In [2]:
##########################################################################################
# Abraham Tishelman-Charny                                                               #
# 3 March 2021                                                                           #
#                                                                                        #
# The purpose of this notebook is to validate Resonant HH UL gridpacks. This can         #
# also be used to study any MadGraph 5 gridpacks.                                        #
##########################################################################################

# Thanks to:
# https://pydoc.net/pylhe/0.0.2/pylhe/ ##-- pylhe 
# https://github.com/scikit-hep/pylhe/blob/master/examples/zpeak.ipynb

In [3]:
##-- Imports 
import pylhe 
import ROOT
import math
from matplotlib import pyplot as plt 
import numpy as np 
import os 
import pandas as pd 
import pickle 
from ROOT import TLorentzVector

Welcome to JupyROOT 6.22/06


In [4]:
##-- Get LHE Files assuming directory tree structure: LHE_Files/<gridpack_Type>/<files>
# gridpack_Type = "GF_Spin-0"
# gridpack_Type = "GF_Spin-2"
gridpack_Type = "VBF_Spin-0"
# gridpack_Type = "VBF_Spin-2"

files = ["LHE_Files/%s/%s"%(gridpack_Type,file) for file in os.listdir('./LHE_Files/%s'%(gridpack_Type)) if file.endswith('.lhe')] 
for file in files:
    print("Found File:",file)
print("N files:",len(files))

Found File: LHE_Files/VBF_Spin-0/VBFToRadionToHH_M1000_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
Found File: LHE_Files/VBF_Spin-0/VBFToRadionToHH_M250_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
Found File: LHE_Files/VBF_Spin-0/VBFToRadionToHH_M3000_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
N files: 3


In [5]:
##-- Create Pandas dataframe for each file 

##-- Parameters 
##################
debug = 0      
maxEvents = 100
lessPoints = 0
##################

##-- Element in lhe file name which has mass information 
massGPtypeDict = {
    "GF_Spin-0" : 3,
    "GF_Spin-2" : 5,
    "VBF_Spin-0" : 1
}
massElement = massGPtypeDict[gridpack_Type]

##-- Functions for Event Loop
def GetPt(p):
    px = p.px
    py = p.py 
    pt = np.sqrt(px**2 + py**2)
    return pt

def GetDR(eta1, eta2, phi1, phi2):
    DR = np.sqrt((eta1 - eta2)**2 + (phi1 - phi2)**2)
    return DR

##-- For each LHE file (one per gridpack)
for i_file, file in enumerate(files):

    ##-- File Printout
    print("On file:",file)
    mass = file.split('/')[-1].split('_')[massElement]
    mass = mass.replace("M", "")
    print("mass:", mass)  
    
    if(lessPoints):
        if(mass != "250" and mass != "3000"): continue 
    
    ##-- Prepare dictionary of lists for pandas dataframe 
    d = {}
    
    ##-- Define particles based on gridpack type 
    particleDictionary = {
        "GF_Spin-0" : ["X","Lead_H","Sublead_H","Lead_g","Sublead_g"], 
        "GF_Spin-2" : ["X","Lead_H","Sublead_H","Lead_g","Sublead_g"],
        "VBF_Spin-0" : ["X","Lead_H","Sublead_H","Lead_Incoming_q","Sublead_Incoming_q", "Lead_Outgoing_q", "Sublead_Outgoing_q"], 
        "VBF_Spin-2" : ["X","Lead_H","Sublead_H","Lead_Incoming_q","Sublead_Incoming_q", "Lead_Outgoing_q", "Sublead_Outgoing_q"]
    }
    
    ##-- Define particles to determine lead / sublead for 
    doubleParticleDict = {
        "GF_Spin-0" : ["H", "g"], 
        "GF_Spin-2" : ["H", "g"], 
        "VBF_Spin-0" : ["H", "Incoming_q", "Outgoing_q"], 
        "VBF_Spin-2" : ["H", "Incoming_q", "Outgoing_q"]
    }    
    
    particle_Names = particleDictionary[gridpack_Type]
    doubleParticles = doubleParticleDict[gridpack_Type]
    
    ##-- Define variables to save for each particle
    variables = ["px", "py", "pz", "e", "m", "spin", "status", "lifetime", "pt"]
    ##-- possible variables: ['id','status','color1','color2','px','py','pz','e','m','lifetime','spin']  
    
    ##-- Initialize dictionary lists 
    for p in particle_Names:
        for v in variables:
            columnName = "%s_%s"%(p,v)
            d[columnName] = [] 
    
    ##-- Save both Higgs' eta and phi 
    for H_particle in ["Lead_H", "Sublead_H"]:
        for H_var in ["eta", "phi"]:
            exec("d['%s_%s'] = []"%(H_particle, H_var))
            
    ##-- For both GF and VBF: Initialize lists to save event by event pdgIds, DR(H, H), Deta(HH), Dphi(HH)
    d["pdgIds"] = []
    d["DR_HH"] = []
    d["HH_Deta"] = []
    d["HH_Dphi"] = []
    
    ##-- VBF Case: Initialize lists to save event by event eta and phi of incoming / outgoing quarks 
    if("VBF" in gridpack_Type):
        for q_particle in ["Lead_Outgoing_q", "Sublead_Outgoing_q", "Lead_Incoming_q", "Sublead_Incoming_q"]:
            for q_var in ["eta", "phi"]:
                exec("d['%s_%s'] = []"%(q_particle, q_var))   
               
        ##-- Save DR(outgoing_quark_1, outgoing_quark_2), DEta and DPhi for VBF case 
        d['DR_OutqOutq'] = []
        d['OutqOutq_Deta'] = []
        d['OutqOutq_Dphi'] = []
        
        ##-- Same for incoming quarks (probably not as interesting (?))
        d['DR_InqInq'] = []
        d['InqInq_Deta'] = []
        d['InqInq_Dphi'] = []         
    
    ##-- For Each Event
    print("Looping Events ...")
    for ie, event in enumerate(pylhe.readLHE(file)):
        if(ie%1000 == 0): print("On Event %s"%(ie))
        if(ie == maxEvents): 
            print("STOPPING at %s events",%(maxEvents))
            break 
        particles = event.particles
        pdgIds = []
        foundFirstHiggs, foundFirstGluon = 0, 0  
        foundFirstIncomingQuark, foundFirstOutgoingQuark = 0, 0
        
        ##-- Get Particles
        for particle in particles:
            pdgId = particle.id
            pdgIds.append(pdgId)
            status = particle.status
            
            ##-- Lots of particle information if debugging
            if(debug): 
                print("")
                print("pdgId:",pdgId)
                print("mass:",particle.m)
                print("mother1:",particle.mother1)
                print("mother2:",particle.mother2)
                print("color1:",particle.color1)
                print("color2:",particle.color2)                
                print("status:",particle.status)
                print("lifetime:",particle.lifetime)  
                print("px:",particle.px)  
                print("py:",particle.py)  
                print("pz:",particle.pz)  
                print("spin:",particle.spin)
            
            ##-- Gluons (GF only)
            if(pdgId == 21 and ("GF" in gridpack_Type) and not foundFirstGluon):
                foundFirstGluon = 1 
                g1 = particle 
            elif(pdgId == 21 and ("GF" in gridpack_Type) and foundFirstGluon):
                g2 = particle  
                
            ##-- Incoming Quarks (VBF only)
            ##-- For some reason incoming VBF quarks are defined as pdgId == 21 in the LHE file. Guessing it's due to gridpack model import 
            elif(pdgId == 21 and status == -1.0 and ("VBF" in gridpack_Type) and not foundFirstIncomingQuark):
                foundFirstIncomingQuark = 1 
                Incoming_q1 = particle
            elif(pdgId == 21 and status == -1.0 and ("VBF" in gridpack_Type) and foundFirstIncomingQuark):
                Incoming_q2 = particle     
                
            ##-- Outgoing Quarks (VBF only)
            ##-- For some reason incoming VBF quarks are defined as pdgId == 21 in the LHE file. Guessing it's due to gridpack model import
            elif(pdgId == 21 and status == 1 and not foundFirstOutgoingQuark):
                foundFirstOutgoingQuark = 1 
                Outgoing_q1 = particle
            elif(pdgId == 21 and status == 1 and foundFirstOutgoingQuark):
                Outgoing_q2 = particle                 
                
            ##-- Intermediate vector bosons are exactly equal and opposite in px, py, pz to outgoing quarks (when there is no ISR and FSR from showering)
            ##-- therefore, information of outgoing quarks gives you information of intermediate vector bosons (in LHE case with no ISR and FSR)
            
            ##-- Spin-0 or Spin-2 Resonant Particle
            if(pdgId == 35 or pdgId == 39):
                X = particle 
                
            ##-- Higgs Bosons 
            if(pdgId == 25 and not foundFirstHiggs):
                foundFirstHiggs = 1 
                H1 = particle 
            elif(pdgId == 25 and foundFirstHiggs):
                H2 = particle 
                
        ##-- Determine Leading, Subleading particles based on pT, save variables 
        for p in doubleParticles:
            p1 = eval("%s1"%(p))
            p2 = eval("%s2"%(p))
            p1_pt = GetPt(p1)
            p2_pt = GetPt(p2)
            if(p1_pt > p2_pt):
                exec("Lead_%s = p1"%(p))
                exec("Sublead_%s = p2"%(p))
            elif(p2_pt > p1_pt):
                exec("Lead_%s = p2"%(p))
                exec("Sublead_%s = p1"%(p)) 
            else:
                exec("Lead_%s = p1"%(p))
                exec("Sublead_%s = p2"%(p))   
            
            ##-- Save variables 
            for v in variables:
                
                ##-- pt gets special treatment
                if(v == "pt"):
                    ##-- Leading variable expression, subleading variable expression
                    exec("d['Lead_%s_%s'].append(p1_pt)"%(p, v))
                    exec("d['Sublead_%s_%s'].append(p2_pt)"%(p, v))
                else:
                    ##-- Leading variable expression, subleading variable expression
                    lvExp = "Lead_%s.%s"%(p, v)
                    slvExp = "Sublead_%s.%s"%(p, v)
                    exec("Lead_%s_%s = %s"%(p, v, lvExp))
                    exec("Sublead_%s_%s = %s"%(p, v, slvExp))
                    exec("d['Lead_%s_%s'].append(Lead_%s_%s)"%(p, v, p, v))
                    exec("d['Sublead_%s_%s'].append(Sublead_%s_%s)"%(p, v, p, v))                    
                
        ##-- Resonant particle
        X_pt = GetPt(X)
        for v in variables:
            c = "X_%s"%(v)
            if(v == "pt"):
                X_pt = GetPt(X)
            else:
                exec("%s = X.%s"%(c, v))
            exec("d['%s'].append(%s)"%(c, c))
        
        ##-- DeltaR between pairs of particles 
        for p_Name in doubleParticles:
            Lead_p_v = TLorentzVector() ##-- Lead particle vector
            Sublead_p_v = TLorentzVector() ##-- Subleading particle vector 
            
            ##-- Set pxpypzE of TLorentzVector in order to get Eta and Phi 
            for rank in ["Lead", "Sublead"]:
                for kinVar in ["px", "py", "pz", "e"]:
                    exec("%s_p_%s = %s_%s_%s"%(rank, kinVar, rank, p_Name, kinVar))

            Lead_p_v.SetPxPyPzE(Lead_p_px, Lead_p_py, Lead_p_pz, Lead_p_e)
            Sublead_p_v.SetPxPyPzE(Sublead_p_px, Sublead_p_py, Sublead_p_pz, Sublead_p_e)

            ##-- Get Eta and Phi 
            Lead_p_eta = Lead_p_v.Eta()
            Lead_p_phi = Lead_p_v.Phi()
            Sublead_p_eta = Sublead_p_v.Eta()
            Sublead_p_phi = Sublead_p_v.Phi()
            
            Deta = float(Lead_p_eta - Sublead_p_eta)
            Dphi = float(Lead_p_phi - Sublead_p_phi)

            for rank in ["Lead", "Sublead"]:
                for ang_var in ["eta", "phi"]:
                    exec("d['%s_%s_%s'].append(%s_p_%s)"%(rank, p_Name, ang_var, rank, ang_var))

            DR = GetDR(Lead_p_eta, Sublead_p_eta, Lead_p_phi, Sublead_p_phi)

            if(p_Name == "H"):
                d['DR_HH'].append(DR)
                d['HH_Deta'].append(Deta)
                d['HH_Dphi'].append(Dphi)
            elif(p_Name == "Outgoing_q"):
                d['DR_OutqOutq'].append(DR)
                d['OutqOutq_Deta'].append(Deta)
                d['OutqOutq_Dphi'].append(Dphi)  
            elif(p_Name == "Incoming_q"):
                d['DR_InqInq'].append(DR)
                d['InqInq_Deta'].append(Deta)
                d['InqInq_Dphi'].append(Dphi)                  
        
        ##-- Append event pdgIds
        d['pdgIds'].append(pdgIds)
        
    ##-- Useful debug is making sure length of each variable list is the same after looping events 
    if(debug):
        for p in particle_Names:
            for v in variables:
                columnName = "%s_%s"%(p,v)
                print("columnName:",columnName)
                print("len(%s):"%(columnName),len(d[columnName]))          
        
    ##-- After event loop, save dataframe
    df = pd.DataFrame(data = d)
    if(debug): print(df)
    pickle.dump(df, open( "Dataframes/%s/%s.p" % (gridpack_Type, mass), "wb" )) ##-- Pickle the dataframe 
                
print("DONE")


On file: LHE_Files/VBF_Spin-0/VBFToRadionToHH_M1000_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
mass: 1000
Looping Events ...
On Event 0
STOPPING at %s events 100
On file: LHE_Files/VBF_Spin-0/VBFToRadionToHH_M250_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
mass: 250
Looping Events ...
On Event 0
STOPPING at %s events 100
On file: LHE_Files/VBF_Spin-0/VBFToRadionToHH_M3000_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.lhe
mass: 3000
Looping Events ...
On Event 0
STOPPING at %s events 100
DONE


In [6]:
##-- To check a dataframe 
df = pickle.load( open( "Dataframes/VBF_Spin-0/1000.p", "rb" ) )
df

,X_px,X_py,X_pz,X_e,X_m,X_spin,X_status,X_lifetime,X_pt,Lead_H_px,...,Lead_Incoming_q_eta,Lead_Incoming_q_phi,Sublead_Incoming_q_eta,Sublead_Incoming_q_phi,DR_OutqOutq,OutqOutq_Deta,OutqOutq_Dphi,DR_InqInq,InqInq_Deta,InqInq_Dphi
0,69.142960,137.712415,619.543122,1186.408642,999.993164,0.0,2.0,0.0,154.095614,-99.572946,...,1.000000e+11,0.0,-1.000000e+11,0.0,3.835765,0.876868,-3.734193,2.000000e+11,2.000000e+11,0.0
1,18.578562,256.684009,-759.661697,1281.922644,1000.003964,0.0,2.0,0.0,257.355480,219.501392,...,1.000000e+11,0.0,-1.000000e+11,0.0,3.182733,0.881101,-3.058341,2.000000e+11,2.000000e+11,0.0
2,-53.947208,-83.676100,76.728479,1007.869481,1000.000820,0.0,2.0,0.0,99.558983,-299.337590,...,1.000000e+11,0.0,-1.000000e+11,0.0,3.023131,-1.371433,2.694159,2.000000e+11,2.000000e+11,0.0
3,-203.251579,-422.264524,-185.872977,1119.895917,999.999785,0.0,2.0,0.0,468.634754,-594.497770,...,1.000000e+11,0.0,-1.000000e+11,0.0,1.970481,0.608526,1.874164,2.000000e+11,2.000000e+11,0.0
4,-34.803293,-261.854642,-295.198655,1075.613804,1000.011843,0.0,2.0,0.0,264.157383,-449.052698,...,1.000000e+11,0.0,-1.000000e+11,0.0,4.151113,-2.436883,3.360556,2.000000e+11,2.000000e+11,0.0
5,30.899009,-160.527198,72.842292,1015.889917,1000.001297,0.0,2.0,0.0,163.473943,-444.790744,...,1.000000e+11,0.0,-1.000000e+11,0.0,3.386763,-1.722315,2.916127,2.000000e+11,2.000000e+11,0.0
6,-80.894140,19.628972,76.467021,1006.369240,1000.001341,0.0,2.0,0.0,83.241567,-448.982956,...,1.000000e+11,0.0,-1.000000e+11,0.0,2.597565,0.015778,-2.597517,2.000000e+11,2.000000e+11,0.0
7,389.572150,-507.565700,-57.663544,1188.575943,999.999144,0.0,2.0,0.0,639.835448,331.615450,...,1.000000e+11,0.0,-1.000000e+11,0.0,0.978627,0.124327,-0.970697,2.000000e+11,2.000000e+11,0.0
8,265.841605,-144.740312,756.106263,1289.696954,1000.000018,0.0,2.0,0.0,302.690463,236.570685,...,1.000000e+11,0.0,-1.000000e+11,0.0,3.965329,1.198350,-3.779919,2.000000e+11,2.000000e+11,0.0
9,446.018867,-119.360539,-558.626207,1235.008471,1000.001458,0.0,2.0,0.0,461.713946,235.459071,...,1.000000e+11,0.0,-1.000000e+11,0.0,5.921853,-2.932275,5.144911,2.000000e+11,2.000000e+11,0.0


In [9]:
##-- Plot variables for mass points together 
##-- All distributions together

##-- Parameters 
##################
ol = "/eos/user/a/atishelm/www/UL_Gridpacks/" ##-- Output location for plots 
gridpack_Type = "VBF_Spin-0"
legendOutside = 1 ##-- Place legend outside of plot axes 
lessPoints = 0 ##-- Only plot a few mass points 
##################

##-- Define nbins, xmin, xmax, units for variables 
def GetVarParams(v):
    VarDict = {
        "Lead_H_pt" : [150, 0, 1500, "GeV"],
        "Sublead_H_pt" : [150, 0, 1500, "GeV"],
        "X_spin" : [4, 0, 4, "unitless"],
        "X_m" : [100, 0, 3100, "GeV"],
        "X_pz" : [60, -3000, 3000, "GeV"],
        "absX_pz" : [30, 0, 3000, "GeV"],
        "DR_HH" : [200, 3, 5, "unitless"],
        "pdgIds" : [40, 0, 40, "unitless"],
        "Lead_H_eta" : [14, -7, 7, "radians"],
        "Lead_H_phi" : [14, -7, 7, "radians"],
        "Sublead_H_eta" : [14, -7, 7, "radians"],
        "Sublead_H_phi" : [14, -7, 7, "radians"],
        "HH_Deta" : [50, -7, 7, "radians"] ,
        "HH_Dphi" : [50, -7, 7, "radians"],
        "absHH_Deta" : [50, 0, 7, "radians"] ,
        "absHH_Dphi" : [50, 0, 7, "radians"],        
        "X_e" : [50, 0, 5000, "GeV"],
        "Lead_g_pz" : [60, -3000, 3000, "GeV"],
        "Lead_g_pt" : [10, 0, 100, "GeV"],
        "Sublead_g_pz" : [60, -3000, 3000, "GeV"],
        "Sublead_g_pt" : [10, 0, 100, "GeV"],
        "Lead_g_px" : [60, -3000, 3000, "GeV"],
        "Sublead_g_px" : [60, -3000, 3000, "GeV"],
        "Lead_g_py" : [60, -3000, 3000, "GeV"],
        "Sublead_g_py" : [60, -3000, 3000, "GeV"],
        "G_pz_diff" : [50, 0, 5000, "GeV"],
        "G_pz_sum" : [60, -4500, 4500, "GeV"],
        "G_E_sum" : [50, 0, 5000, "GeV"],
    }
    return VarDict[v]

##-- Get dataframe files 
df_names_ = []
prefix = "Dataframes/%s"%(gridpack_Type)
df_names_ = ["%s/%s"%(prefix, file) for file in os.listdir(prefix)]

##-- Order by mass for plotting order purposes  
df_names = sorted(df_names_, key = lambda x: (float(x.split('/')[-1].split('.')[0])))

##-- Define Validation Variables based on gridpack type 
GF_Vars = ["pdgIds", "X_m", "G_E_sum", "Lead_H_pt", "Sublead_H_pt", "absHH_Deta", "absHH_Dphi"]
VBF_Vars = ["pdgIds", "X_m", "Lead_H_pt", "Sublead_H_pt", "absHH_Deta", "absHH_Dphi"]
ValVarsDict = {
    "GF_Spin-0" : GF_Vars, 
    "GF_Spin-2" : GF_Vars,
    "VBF_Spin-0" : VBF_Vars,
    "VBF_Spin-2" : VBF_Vars
}

variables = ValVarsDict[gridpack_Type]

##-- To hold values for avg +/- stdev vs. Mass plots
masses = []
N_Gluons_vals, N_Higgs_vals = [], [] 
N_Events = 10000 ##-- This is assumed based on LHE file production 

##-- Initialize mean and stdev lists to plot 
for var in variables:
    exec("%s_mean = []"%(var))
    exec("%s_stdev = []"%(var))

##-- For each variable, plot all mass point distributions and save mean and stdev values for var vs. mass point plots 
for v in variables:
    print("On variable:",v)
    
    ##-- Get variable dependent parameters 
    nbins, xmin, xmax, unit = GetVarParams(v)
    allBins = np.linspace(xmin, xmax, nbins + 1)
    
    ##-- Setup Colors 
    cm = plt.get_cmap('jet') ##-- Choose nice color scheme: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    NUM_COLORS = len(df_names)
    fig, ax = plt.subplots()
    colors = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]
    colorsDoubled = [] ##-- to add error bars and hist without incrementing color 
    for ic, c in enumerate(colors):
        colorsDoubled.append(c) ##-- The purpose of doubling colors in the cycle is because both a hist and errorbar are being plotted per mass point. Want same color for both 
        colorsDoubled.append(c)
    ax.set_prop_cycle(color = colorsDoubled)
    
    ##-- For each dataframe (one per LHE file)
    for i, df_name in enumerate(df_names):
        
        ##-- Dataframe info 
        endPath = df_name.split('/')[-1]
        if(lessPoints):
            if(endPath != "250.p" and endPath != "1000.p" and endPath != "3000.p"): continue
#         colors.append(color)
        df = pickle.load( open( df_name, "rb" ) ) ##-- Load dataframe 
        plotLabel = df_name.split('/')[-1]
        plotLabel = plotLabel.replace(".p","")  
        masses.append(plotLabel)
        
        ##-- Create numpy histogram normalized to integral = 1 
        
        ##-- Special cases 
        if(v=='pdgIds' or v =='N_Gluons' or v == 'N_Higgs'):
            allVals = []
            for vals in df['pdgIds']:
                for val in vals:
                    allVals.append(val)
            nonNormyvals, binedges = np.histogram(allVals, bins=nbins, range=(xmin,xmax))
            yvals, binedges = np.histogram(allVals, bins=nbins, range=(xmin,xmax),density=False)     
        elif(v == "G_pz_diff"):
            v1 = df["Lead_g_pz"]
            v2 = df["Sublead_g_pz"]
            all_df_vals = np.subtract(v1, v2)
            nonNormyvals, binedges = np.histogram(all_df_vals, bins = nbins, range = (xmin, xmax))
            yvals, binedges = np.histogram(all_df_vals, bins=nbins, range=(xmin,xmax),density=True)  
        elif(v == "G_pz_sum"):
            v1 = df["Lead_g_pz"]
            v2 = df["Sublead_g_pz"]
            all_df_vals = np.add(v1, v2)
            nonNormyvals, binedges = np.histogram(all_df_vals, bins = nbins, range = (xmin, xmax))
            yvals, binedges = np.histogram(all_df_vals, bins=nbins, range=(xmin,xmax),density=True)
        elif(v == "G_E_sum"):
            v1 = df["Lead_g_e"]
            v2 = df["Sublead_g_e"]
            all_df_vals = np.add(v1, v2)
            nonNormyvals, binedges = np.histogram(all_df_vals, bins = nbins, range = (xmin, xmax))
            yvals, binedges = np.histogram(all_df_vals, bins=nbins, range=(xmin,xmax),density=True)            
        elif(v == "G_X_pz_diff"):
            v1 = df["Lead_g_pz"]
            v2 = df["Sublead_g_pz"]
            X_pz = df["X_pz"]
            G_pz_sum = np.add(v1, v2)
            all_df_vals = np.subtract(G_pz_sum, X_pz)
            nonNormyvals, binedges = np.histogram(all_df_vals, bins = nbins, range = (xmin, xmax))
            yvals, binedges = np.histogram(all_df_vals, bins=nbins, range=(xmin,xmax),density=True)            
            
        ##-- Non-Special cases 
        else:
            if("abs" in v):
                v_no_abs = v[:].replace("abs", "")
                all_df_vals = all_df_vals = abs(df[v_no_abs])
            else:
                v_no_abs = v[:].replace("abs", "")
                all_df_vals = df[v_no_abs]
            if("abs" in v):
                nonNormyvals, binedges = np.histogram(all_df_vals, bins = nbins, range = (xmin, xmax))
                yvals, binedges = np.histogram(all_df_vals, bins=nbins, range=(xmin,xmax),density=True)                
            else:
                nonNormyvals, binedges = np.histogram(all_df_vals, bins = nbins, range = (xmin, xmax))
                yvals, binedges = np.histogram(all_df_vals, bins=nbins, range=(xmin,xmax),density=True)                

        ##-- Save mean and stdev of histogram (except for pdgIds variable since it's a list per event)
        if(v != "pdgIds"):
            mean, stdev = np.average(all_df_vals), np.std(all_df_vals)
            exec("%s_mean.append(mean)"%(v))
            exec("%s_stdev.append(stdev)"%(v))
        
        ##-- Get Poissonian bin uncertainties and scale to normalized bin height
        nonNormYerrors = np.sqrt(nonNormyvals)
        relErrors = np.divide(nonNormYerrors,nonNormyvals, out=np.zeros_like(nonNormYerrors),where=nonNormyvals!=0) 
        errors = yvals * relErrors
        bincenters = 0.5*(binedges[1:]+binedges[:-1])
        width = float(binedges[1])-float(binedges[0])
        
        ##-- Save number of gluons and higgs bosons per event 
        ##-- Note that for VBF pdgId == 21 for incoming / outgoing quarks
        if(v=='pdgIds'):
            for ic,center in enumerate(bincenters):
                val = yvals[ic]
                if(val!=0): 
                    pdgId = int(center - width / 2)
#                     print("pdgId:",pdgId,": ",val)
                    if(pdgId == 21):
                        N_Gluons_vals.append(val)
                    elif(pdgId == 25):
                        N_Higgs_vals.append(val)
                        
        ##-- Define x errors as bin width / 2 for visual purposes 
        xerrors = []
        for bin_i in range(0,nbins): xerrors.append(width/2)
        xerrors_a = np.array(xerrors) # xerrors array 
        binedges = np.delete(binedges,0,0)  
        
        ##-- Plot normalized histogram 
        plt.hist(bincenters,
                 weights = yvals,
                 histtype = 'stepfilled',
                 edgecolor = 'black',
                 bins = allBins,
                 label=plotLabel,
                 zorder = i
#                  zorder = float(float(NUM_COLORS) - float(i)) ##-- For opposite mass point z-ordering in plot
        )           
        
        ##-- Add poissonian bin uncertainties 
        plt.errorbar(
                    bincenters,
                    yvals,
                    yerr=errors,
                    ecolor = 'black',
                    linestyle = '',
                    fmt = 'none',
#                     zorder = float(float(NUM_COLORS) - float(i)) ##-- For opposite mass point z-ordering in plot
                    zorder = i + 0.5
        )        
        
    ##-- Decorate plot 
    plt.ylabel("Entries [A.U.]")
    if(v == "Lead_H_pt" or v == "Sublead_H_pt"):
        plt.title("Higgs pT")
        xlabel = "%s [%s]"%("Higgs pT",unit)
    else:
        plt.title(v)
        xlabel = "%s [%s]"%(v,unit)
    plt.xlabel(xlabel)
    plt.tick_params(direction='in')
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both') 
    if(v == "DR_HH" or v == "absHH_Deta"): plt.yscale('log')
    if(legendOutside):
        plt.legend(title = "Mass [GeV]", bbox_to_anchor=(1.01, 1.05), loc='upper left')
        fig.set_size_inches(13, 6)
    else:
        leg = plt.legend(title = "Mass [GeV]")
        fig.set_size_inches(10, 10)
    fig.tight_layout()

    ##-- Save plot to output location as png and pdf assuming gridpack_Type directory exists in output location 
    outName1 = "%s/%s/%s.png"%(ol,gridpack_Type,v)
    outName2 = "%s/%s/%s.pdf"%(ol,gridpack_Type,v)
    plt.savefig(outName1)
    plt.savefig(outName2)
    plt.close()         
            
print("DONE")


On variable: pdgIds
On variable: X_m
On variable: Lead_H_pt
On variable: Sublead_H_pt
On variable: absHH_Deta
On variable: absHH_Dphi
DONE


In [12]:
##-- Produce variable average vs. Mass point version of plots

# ##-- Plot number of particles per event 
if("pdgIds" in variables):
#     N_Gluons_vals_perE = [val/N_Events for val in N_Gluons_vals] 
    N_Higgs_vals_perE = [val/N_Events for val in N_Higgs_vals]  

#     mass_G_pairs = sorted(list(zip(masses, N_Gluons_vals_perE)), key = lambda x: (float(x[0])))
    mass_H_pairs = sorted(list(zip(masses, N_Higgs_vals_perE)), key = lambda x: (float(x[0])))

    orderedMasses = [i[0] for i in mass_H_pairs]
#     GluonsPerEvent_values = [i[1] for i in mass_G_pairs]
    HiggsPerEvent_values = [i[1] for i in mass_H_pairs]

    x_pos = [i for i, _ in enumerate(orderedMasses)]
    y_pos = np.arange(len(orderedMasses))

    ##-- Particles Per Event vs. Mass plots 
#     for p in ["Gluons", "Higgs"]:
    for p in ["Higgs"]:
        fig, ax = plt.subplots()
        fig.set_size_inches(14, 7)
        vals = eval("%sPerEvent_values"%(p))
        plt.bar(y_pos, vals, align='center', color = 'dodgerblue', edgecolor = 'black')
        plt.xticks(y_pos, orderedMasses, fontsize = 12)
        plt.xlabel('Resonant Mass [GeV]', fontsize = 20)
        plt.yticks([0, 1, 2], ["0","1","2"], fontsize = 20)
        plt.title("%s Per Event"%(p), fontsize = 35)
        fig.tight_layout()
        outName1 = "%s/%s/%s.png"%(ol,gridpack_Type,"%s_Per_Event"%(p))
        outName2 = "%s/%s/%s.pdf"%(ol,gridpack_Type,"%s_Per_Event"%(p))
        print("Saving:",outName1)
        print("Saving:",outName2)
        plt.savefig(outName1)
        plt.savefig(outName2)
        plt.close()

##-- Plot bar graphs - One bin per mass point. X axis does not scale with variable
print("Plotting Bar Graphs:")
for v in variables:
    if(v == "pdgIds"): continue 
    unit = GetVarParams(v)[3]
    print("%s vs. Mass"%(v))

    ##-- Get variable mean and stdev values per mass point 
    exec("mean_vals = %s_mean"%(v))
    exec("stdev_vals = %s_stdev"%(v))

    ##-- Sort values by increasing resonant mass value 
    exec("massMeanPairs_Sorted = sorted(list(zip(masses, mean_vals)), key = lambda x: (float(x[0])))")
    exec("massStdevPairs_Sorted = sorted(list(zip(masses, stdev_vals)), key = lambda x: (float(x[0])))")
    orderedMasses = [i[0] for i in massMeanPairs_Sorted]
    orderedMeans = [i[1] for i in massMeanPairs_Sorted]
    orderedStdevs = [i[1] for i in massStdevPairs_Sorted] 
    orderedStddevs_oneSide = [i / 2. for i in orderedStdevs] ##-- Divide by 2 to define symmetric uncertainty
    x_pos = [i for i, _ in enumerate(orderedMasses)]
    y_pos = np.arange(len(orderedMasses))   
    
    ##-- Prepare Plot 
    fig, ax = plt.subplots()
    fig.set_size_inches(14, 7) ##-- Wide because of the many resonant mass points 
    plt.bar(y_pos, orderedMeans, color = 'dodgerblue', align='center', edgecolor = 'black') 
    plt.errorbar(y_pos, orderedMeans, yerr=orderedStddevs_oneSide, ecolor = 'black', linestyle = '')
    plt.scatter(y_pos, orderedMeans, color = 'black')
    plt.xticks(y_pos, orderedMasses, fontsize = 10)
    plt.xlabel('Resonant Mass [GeV]', fontsize = 20)
    plt.yticks(fontsize = 20)
    if(v == "X_spin"): plt.yticks([-1, 0, 1], ["-1","0","1"], fontsize = 20)
    if((v == "Lead_H_pt" or v == "Sublead_H_pt") and "GF" in gridpack_Type):
        vTitle = "Higgs pt"
    else: 
        vTitle = v
    plt.title("Average %s"%(vTitle), fontsize = 35)
    plt.ylabel("Average %s [%s]"%(vTitle, unit), fontsize = 20)
    fig.tight_layout()
    outName1 = "%s/%s/%s.png"%(ol,gridpack_Type,"BarGraph_%s_vs_mass"%(vTitle))
    outName2 = "%s/%s/%s.pdf"%(ol,gridpack_Type,"BarGraph_%s_vs_mass"%(vTitle))

    plt.savefig(outName1)
    plt.savefig(outName2)
    plt.close()    
    
print("---------------------")
print("Plotting 2d Scatters:")
##-- 2D graph: Variable vs. mass point, x axis is properly scaled to mass value
for v in variables:
    if(v == "pdgIds"): continue 
    print("%s vs. Mass"%(v))
    unit = GetVarParams(v)[3]    

    ##-- Get variable mean and stdev values per mass point 
    exec("mean_vals = %s_mean"%(v))
    exec("stdev_vals = %s_stdev"%(v))

    ##-- Sort values by increasing resonant mass value 
    exec("massMeanPairs_Sorted = sorted(list(zip(masses, mean_vals)), key = lambda x: (float(x[0])))")
    exec("massStdevPairs_Sorted = sorted(list(zip(masses, stdev_vals)), key = lambda x: (float(x[0])))")
    orderedMasses = [i[0] for i in massMeanPairs_Sorted]
    orderedMeans = [i[1] for i in massMeanPairs_Sorted]
    orderedStdevs = [i[1] for i in massStdevPairs_Sorted] 
    orderedStddevs_oneSide = [i / 2. for i in orderedStdevs] ##-- Divide by 2 to define symmetric uncertainty
    x_pos = [i for i, _ in enumerate(orderedMasses)]
    y_pos = np.arange(len(orderedMasses))   
    fig, ax = plt.subplots()
    fig.set_size_inches(14, 7)   
    orderedMasses_vals = [float(val) for val in orderedMasses]
    if(v == "X_m" or v == "G_E_sum"): 
        plt.plot([0, 1], [0, 1] , linestyle = "--", transform=ax.transAxes)
        plt.xlim(0, 3500)
        plt.ylim(0, 3500)        
    plt.scatter(orderedMasses_vals, orderedMeans, color = 'black')
    plt.errorbar(orderedMasses_vals, orderedMeans, yerr=orderedStddevs_oneSide, ecolor = 'black', linestyle = '')
    if(v == "Lead_H_pt" or v == "Sublead_H_pt"):
        vTitle = "Higgs pt"
        ax.plot([0, 3100], [0, 1550], ls="--", c=".3")
    else: 
        vTitle = v[:]
    plt.title("Average %s"%(vTitle), fontsize = 35)
    plt.ylabel("Average %s [%s]"%(vTitle, unit), fontsize = 20)
    plt.xlabel('Resonant Mass [GeV]', fontsize = 20)
    if(v != "X_m" and v != "G_E_sum"): 
        plt.xlim(0, 3100)
    fig.tight_layout()
    
    outName1 = "%s/%s/%s.png"%(ol,gridpack_Type,"2dGraph_%s_vs_mass"%(vTitle))
    outName2 = "%s/%s/%s.pdf"%(ol,gridpack_Type,"2dGraph_%s_vs_mass"%(vTitle))

    plt.savefig(outName1)
    plt.savefig(outName2)
    plt.close()     
    
print("DONE")

Saving: /eos/user/a/atishelm/www/UL_Gridpacks//VBF_Spin-0/Higgs_Per_Event.png
Saving: /eos/user/a/atishelm/www/UL_Gridpacks//VBF_Spin-0/Higgs_Per_Event.pdf
Plotting Bar Graphs:
X_m vs. Mass
Lead_H_pt vs. Mass
Sublead_H_pt vs. Mass
absHH_Deta vs. Mass
absHH_Dphi vs. Mass
---------------------
Plotting 2d Scatters:
X_m vs. Mass
Lead_H_pt vs. Mass
Sublead_H_pt vs. Mass
absHH_Deta vs. Mass
absHH_Dphi vs. Mass
DONE
